# Imports

In [1]:
import pandas as pd

# Clean data

In [47]:
df = pd.read_csv('data/paris_apart.csv', low_memory=False)

nan_values = df.isnull().sum().sum()
print(f"Nombre de valeurs: {df.size:_}".replace("_", " "))
print(f"Valeurs manquantes : {nan_values:_}".replace("_", " "))
print(f"Qualité des données : {100 - round((nan_values / df.size) * 100, 2)} %")
analysis = {
  'Manquant': df.isnull().sum(),
  'Manquant %': round((df.isnull().sum() / len(df)) * 100, 2),
  'Type': df.dtypes
}
pd.DataFrame(analysis).sort_values('Manquant %', ascending=False)

# Drop duplicates lines & filters to get vente no disposition
df.drop_duplicates(inplace=True)
df = df[(df['No disposition'] == 1) & (df['Nature mutation'] == 'Vente')]

# Drop NaN, price missing and selected columns
df = df.dropna(axis=1, thresh=0)
df = df[df['Valeur fonciere'].notna()]
df = df.drop(['id', 'Type local','Code type local','Code departement','Commune','B/T/Q','Code voie','Nature mutation'], axis=1)

# Format values on selected columns
for value in df[['Valeur fonciere','1er lot','Surface Carrez du 1er lot','2eme lot','Surface Carrez du 2eme lot','3eme lot','Surface Carrez du 3eme lot','Surface Carrez du 4eme lot','Surface Carrez du 5eme lot']]: 
    df[f'{value}'] = df[f'{value}'].replace(',','.', regex=True)
    df[f'{value}'] = df[f'{value}'].replace('[a-zA-Z]+','', regex=True)

# Convert value type
df=df.astype({'Valeur fonciere': 'float64','1er lot':'float64',
            'Surface Carrez du 1er lot':'float64','2eme lot':'float64','Surface Carrez du 2eme lot':'float64',
            '3eme lot':'float64','Surface Carrez du 3eme lot':'float64','Surface Carrez du 4eme lot':'float64','Surface Carrez du 5eme lot':'float64'},errors='raise')

# Calculate area of the apartment
df['surface']=df['Surface Carrez du 1er lot'].fillna(0)+df['Surface Carrez du 2eme lot'].fillna(0)+df['Surface Carrez du 3eme lot'].fillna(0)+df['Surface Carrez du 4eme lot'].fillna(0)+df['Surface Carrez du 5eme lot'].fillna(0)
df=df.drop(columns=['1er lot','2eme lot','3eme lot','4eme lot','5eme lot','Surface Carrez du 1er lot','Surface Carrez du 2eme lot','Surface Carrez du 3eme lot','Surface Carrez du 4eme lot','Surface Carrez du 5eme lot'],axis=1)

# Check value types
for value in df:
  if df[f'{value}'].dtypes=='int64' or df[f'{value}'].dtypes=='float64' :
      df[f'{value}'].fillna(0, inplace=True)
  elif df[f'{value}'].dtypes=='object':
      df[f'{value}'].fillna('', inplace=True)

print(df.columns)
df.head()

Nombre de valeurs: 8 953 956
Valeurs manquantes : 4 388 710
Qualité des données : 50.99 %
Index(['Code service sages', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Valeur fonciere', 'No voie',
       'Type de voie', 'Voie', 'Code postal', 'Code commune',
       'Prefixe de section', 'Section', 'No plan', 'No Volume',
       'Nombre de lots', 'Identifiant local', 'Surface reelle bati',
       'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'surface'],
      dtype='object')


/var/folders/b3/tn1zt4lx1j1_0hhxtfhfgwp80000gn/T/ipykernel_23021/974869225.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[f'{value}'].fillna(0, inplace=True)
/var/folders/b3/tn1zt4lx1j1_0hhxtfhfgwp80000gn/T/ipykernel_23021/974869225.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

,Code service sages,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Valeur fonciere,...,No plan,No Volume,Nombre de lots,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,surface
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,04/07/2017,571110.0,...,35,0.0,2,0.0,42.0,3.0,,,0.0,49.51
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,04/07/2017,1578150.0,...,44,0.0,3,0.0,135.0,4.0,,,0.0,139.02
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,05/07/2017,2810880.0,...,60,0.0,1,0.0,220.0,6.0,,,0.0,219.60
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,03/07/2017,130000.0,...,81,0.0,2,0.0,14.0,1.0,,,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,05/07/2017,485000.0,...,78,0.0,1,0.0,38.0,3.0,,,0.0,50.07
